# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [1]:
import json

file = "/data/negotiated_rates.json"

#import the json file, open and read it.
negotiation = json.load(open(file))

def average_rate(file):
    """(str) -> float
    This function takes the JSON file 'negotitated_rates' as its input parameter and calculates an average allowed amounts of all of the rates in the file and returns a float(rounded to two decimal places) as a result.
    
    
    >>> average_rate('/data/negotiated_rates.json')
    38.67
    """
    
    total_allowed = 0
    count = 0
    
  
    for network in negotiation['out_of_network']:
        allowed = network['allowed_amounts']
        for amt in allowed:
            amount = amt['payments']['allowed_amount']
            total_allowed += amount
            count += 1
   
   
    allowed_amount = round(total_allowed/count,2)
        
        
    return allowed_amount
        
    
    
    
    



In [2]:
average_rate(file)

38.67

In [3]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

In [4]:
import doctest
doctest.run_docstring_examples(average_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    average_rate('/data/negotiated_rates.json')
Expecting:
    38.67
ok


---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [5]:
import json

file = "/data/negotiated_rates.json"

negotiation = json.load(open(file))

def get_rate(file, billing_code, service_code):
    """(str, str, str)->float or None
    This function will take json file called 'negotiated_rates' as an input to look up the allowed amount for any given billing code
    and service code combination. It will return the amount value as a float and if a combination is not found, None is returned.
    
    >>> get_rate('/data/negotiated_rates.json', '97110', '11')
    21.51
    
    >>> get_rate('/data/negotiated_rates.json', '97140', '13')
    """
    
    for network in negotiation['out_of_network']:
        billingcode = network['billing_code']
        allowed = network['allowed_amounts']
        
        for amt in allowed:
            servicecode = amt['service_code']
            amount = amt['payments']['allowed_amount']
            
            if billing_code in billingcode and service_code in servicecode:
                return amount
        

    return None
      
        
    
   

    

In [6]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

In [7]:
import doctest
doctest.run_docstring_examples(get_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_rate('/data/negotiated_rates.json', '97110', '11')
Expecting:
    21.51
ok
Trying:
    get_rate('/data/negotiated_rates.json', '97140', '13')
Expecting nothing
ok


---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [245]:
from datetime import datetime
import json


file = "/data/negotiated_rates.json"

negotiation = json.load(open(file))


def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    """(str, str, str, int, str)->float or None
    This function takes the get_rate() function to calculate the adjusted rate based on the below special rules and also the datetime format is set.
    1. If visit dates are on Mondays, 75% of the allowed amount is charged(amount is rounded to two decimals); 
    2. If the patient's age is 65 or higher, 50% of the allowed amount is charged(amount is rounded to two decimals)
    3. If visit dates are on Mondays and the patient is above or equal to 65,50% of the allowed amount is charged(amount is rounded to two decimals);
    4. If none of the above conditions are met, then charge the whole amount or return the same value
    The function returns a float value or None if the rate is not found in the file
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07')
    8.78
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03')
    6.58
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','12',76,'2022-10-22')
    
    """
    
    
    rate = get_rate(file,billing_code,service_code)
      
    # convert from string format to datetime format
    # and get the date
    d = datetime.strptime(visit_date, "%Y-%m-%d")
    i = d.weekday()
    adjusted_rate = 0  
    
    if rate == None:
        return None
    else:
        if i == 0 and int(pat_age) >= 65:
            adjusted_rate = (rate * 0.5)
        elif i == 0:
            adjusted_rate = (rate * 0.75)
        elif int(pat_age) >= 65:
            adjusted_rate = (rate * 0.5)
        elif i != 0 and int(pat_age) != 65:
            adjusted_rate = rate
        else:
            None
        
    return round(adjusted_rate, 2)
        
   
   
    
    
   
   
        
    

In [246]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

In [247]:
import doctest
doctest.run_docstring_examples(get_adjusted_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07')
Expecting:
    8.78
ok
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03')
Expecting:
    6.58
ok
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','12',76,'2022-10-22')
Expecting nothing
ok


In [248]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03')

6.58

---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [249]:
import json
import csv
from datetime import datetime



def summarize_reimbursement(visits, negotiated_rates): 
    """(str, str)-> dict, dict
    This function takes input parameters from a csv file named 'visits' and the json file('negotiated_rates') and gets the information by using the 
    get_adjusted_rate() function. 
    - The CSV file, json file is imported along with datetime module. Once the files are open and read, two empty dictionaries are created to where the total expected reimbursement by month and hospital will be stored.
    - Adjusted rates for each month of the year is added(rounding it to two decimals), the date is taken from the input csv file to output only the days of the month. This is then stored in the by_month dictionary.
    - For each hospital the adjusted rates are added up (rounding it to two decimals) and placed them in the by_hospital dictionary.
    - The function will then compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information
    1. Total expected reimbursement by month
    2. Total expected reimbursement by hospital
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07']
    277.67
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] 
    378.72
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis']
    380.07
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield']
    518.59
    
    
    """
    
    by_month = {}
    by_hospital = {}
    file = "/data/negotiated_rates.json"
    negotiation = json.load(open(file))
    
    with open(visits) as f:
        visit_file = csv.reader(f)
        next (visit_file, None)
       
        for row in visit_file:
            hospital = row[1]
            pat_age = row[3]
            billing_code = row[4]
            service_code = row[5]
            visit_date = row[6]
            total_adj_rate = get_adjusted_rate(file, row[4], row[5], int(row[3]), row[6])
           #total_adj_rate = get_adjusted_rate(file,billing_code,service_code, int(pat_age), visit_date)
            
            if total_adj_rate == None:
                pass
            else:
                if visit_date[:7] in by_month.keys():
                    by_month[visit_date[:7]] += total_adj_rate
                    by_month[visit_date[:7]] = round(by_month[visit_date[:7]],2)
                else:
                    by_month[visit_date[:7]]= total_adj_rate
            
                if hospital in by_hospital.keys():
                    by_hospital[hospital] +=  total_adj_rate
                    by_hospital[hospital]= round(by_hospital[hospital],2)
                else:
                    by_hospital[hospital]= total_adj_rate
        
    return by_month, by_hospital
        
    
    
        

            

In [250]:
import doctest
doctest.run_docstring_examples(summarize_reimbursement, globals(), verbose=True)

Finding tests in NoName
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07']
Expecting:
    277.67
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] 
Expecting:
    378.72
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis']
Expecting:
    380.07
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield']
Expecting:
    518.59
ok


In [251]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

({'2021-05': 192.38,
  '2021-03': 378.72,
  '2021-07': 277.67,
  '2021-06': 236.53,
  '2021-11': 229.7,
  '2021-10': 234.52,
  '2021-12': 297.87,
  '2021-04': 337.7,
  '2021-09': 160.4,
  '2021-01': 111.91,
  '2021-02': 158.55,
  '2021-08': 152.28},
 {'Missouri Baptist': 514.18,
  'SSM DePaul': 460.02,
  'SLU Hospital': 409.67,
  'Barnes Jewish': 485.7,
  'Mercy Springfield': 518.59,
  'Mercy St. Louis': 380.07})

In [252]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)

In [253]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07

True